In [8]:
import torch
import torch.nn.functional as F

import numpy as np

In [10]:
class conv_block(torch.nn.Module):
    def __init__(self,D_in,D_out, k):
        super(conv_block, self).__init__() 
        p = np.floor((kernel_size - 1) / 2).astype(np.int32)
        self.conv_layer = torch.nn.Conv2d(in_channels= D_in, out_channels= D_out, kernel_size = k ,stride = 2, padding = p) 
        self.conv_layerb = torch.nn.Conv2d(in_channels= D_out, out_channels= D_out, kernel_size = k ,stride = 1, padding = p)
        self.normalize = torch.nn.BatchNorm2d()
        
    def forward(self, x):
        x = F.elu(conv_layer(x)) 
        x = normalize(x)
        x = F.elu(conv_layerb(x))
        x = normalize(x)
        return x 

In [12]:
class upconv(torch.nn.Module):
    def __init__(self,D_in,D_out,k, scale):
        up = nn.Upsample(scale_factor=scale, mode='bilinear')
        conv = nn.torch.nn.Conv2d(in_channels= D_in, out_channels= D_out, kernel_size = k ,stride = 2, padding = p)
        
    def forward(self,x):
        #x = nn.functional.interpolate(x, scale_factor=self.scale, mode='bilinear', align_corners=True)
        x = up(x)
        x = conv(x)
        return torch.nn.functional.elu(x)
    
class iconv(torch.nn.Module): 
    def __init__U(self,D_in,D_out, k):
        self.conv = nn.torch.nn.Conv2d(in_channels= D_in, out_channels= D_out, kernel_size = k ,stride = 1, padding = p)
    
    def forward(self,x):
        x = self.conv(x)
        return torch.nn.functional.elu(x)    

In [17]:
class disp(torch.nn.Module):
    def __init__ (self,D_in, D_out,k ,scale):
        self.conv =  nn.torch.nn.Conv2d(in_channels= D_in, out_channels= D_out, kernel_size = k ,stride = 1, padding = p)
        
    def forward(self, x):
        return self.conv 

In [18]:
class MonoDepthModel(torch.nn.Module):
    def __init__(self):
        
        #encoder 
        self.conv1 = conv_block(D_in=3,D_out=32,k=7)    # h,w /2 
        self.conv2 = conv_block(D_in=32,D_out=64,k=5)   # h,w /4 
        self.conv3 = conv_block(D_in=64,D_out=128,k=3)  # h,w /8
        self.conv4 = conv_block(D_in=128,D_out=256,k=3) # h,w /16
        self.conv5 = conv_block(D_in=256,D_out=512,k=3) # h,w /32
        self.conv6 = conv_block(D_in=512,D_out=512,k=3) # h,w /64
        self.conv7 = conv_block(D_in=512,D_out=512,k=3) # h,w /128 
        
        #Decoder
        self.upconv7 = upconv(D_in = 512, D_out = 512, k = 3) #h,w / 64
        self.iconv7 = iconv(D_in = 1024, D_out = 512, k = 3)
        self.upconv6 = upconv(D_in = 512, D_out = 512, k = 3) #h,w / 32
        self.iconv6 = iconv(D_in = 1024, D_out = 512, k = 3)
        self.upconv5 = upconv(D_in = 512, D_out = 256, k = 3) #h,w / 16
        self.iconv5 = iconv(D_in = 512, D_out = 256, k = 3)
        #-------------------------------------------------------------#
        self.upconv4 = upconv(D_in = 256, D_out = 128, k = 3) #h,w / 8
        self.iconv4 = iconv(D_in = 128, D_out = 128, k = 3)
        self.disp4 = disp(D_in = 128, D_out = 2, k = 3)
        #-------------------------------------------------------------#
        self.upconv3 = upconv(D_in = 128, D_out = 64, k = 3) #h,w / 4
        self.iconv3 = iconv(D_in = 130, D_out = 64, k = 3)   #64+64+2
        self.disp3 = disp(D_in = 64, D_out = 2, k = 3)
        #-------------------------------------------------------------#
        self.upconv2 = upconv(D_in = 64, D_out = 32, k = 3) #h,w / 2
        self.iconv2 = iconv(D_in = 66, D_out = 32, k = 3)   #32+32+2
        self.disp2 = disp(D_in = 64, D_out = 2, k = 3)
        #-------------------------------------------------------------#
        self.upconv1 = upconv(D_in = 32, D_out = 16, k = 3) #h,w / 1
        self.iconv1 = iconv(D_in = 18, D_out = 16, k = 3)
        self.disp1 = disp(D_in = 16, D_out = 2, k = 3)
        
        
    def forward(self,x):
        #encoder 
        x1 = self.conv1(x)  #conv1b
        x2 = self.conv2(x1) #conv2b
        x3 = self.conv3(x2) #conv3b
        x4 = self.conv4(x3) #conv4b
        x5 = self.conv5(x4) #conv5b 
        x6 = self.conv6(x5) #conv6b
        x7 = self.conv7(x6) #conv7b
        
        #decoder 
        upconv7 = self.upconv7(x7)
        iconv7 = self.iconv(torch.cat[upconv7,x6],1)
        
        upconv6 = self.upconv7(iconv7)
        iconv6 = self.iconv(torch.cat[upconv6,x5],1)
        
        upconv5 = self.upconv6(iconv6)
        iconv5 = self.iconv(torch.cat[upconv5,x4],1)
        
        
        upconv4 = self.upconv6(iconv5)
        iconv4 = self.iconv(torch.cat[upconv4,x3],1)
        self.disp4 = self.disp(iconv4)
        
        upconv3 = self.upconv6(iconv5)
        iconv3 = torch.cat[upconv3,x2,upsample_disp(self.disp4)]
        disp3 = self.disp(iconv3)
        
        
        
        
        
        
        
        #concat6 = torch.cat((upconv6, skip5), 1)
        
        return [self.disp1,self.disp2,self.disp3,self.disp4]
    
    def upsample_disp(d):
        return nn.functional.interpolate(d,scale_factor = 2, mode = 'billinear')